In [25]:
from program.setup import *
df = pd.read_csv('Ocean_Commitments_Full.csv', encoding='latin-1')
df.columns = [x.strip().replace(" ","_").replace(".","_").lower() for x in df.columns]
item_idx = df['count']
#display(df)

def uniq(A):
    return np.vstack(np.unique(A,return_counts=True)).T

def pad(x):
    x = str(x)
    if(len(x) == 0):
        return '00'
    elif(len(x) == 1):
        if x.isdigit() == True:
            return x.zfill(2)
        else:
            return x*2
    else:
        return x    

sdgs = df['sdgs'].copy()
sdgs.replace({'Goal ':''}, regex=True, inplace=True)
sdgs = sdgs.str.rstrip(',').str.split(",",expand=True).fillna(0).astype('str')
sdgs = sdgs.applymap(pad)
#display(uniq(sdgs))
#display(sdgs)


targets = df['sdg_tagets'].copy()
targets.replace({' ':''}, regex=True, inplace=True)
targets = targets.str.rstrip(',').str.split(",",expand=True).fillna(0.0).astype('str')
#display(uniq(targets))
#display(targets)

def f(A,n=0):
    B = A.str.split(".").str[n].fillna(0).astype('str')
    return B.apply(pad)
targets_main = targets.apply(f,n=0)#.astype('int')
targets_sub = targets.apply(f,n=1)#.astype('object')
#display(uniq(targets_main))
#display(targets_main)
#display(uniq(targets_sub))
#display(targets_sub)

from sklearn.preprocessing import MultiLabelBinarizer
goals = np.union1d(np.unique(sdgs),np.unique(targets_main))
mlb = MultiLabelBinarizer(classes=goals)

sdg_mlb = mlb.fit_transform(sdgs.values)
sdg_mlb = pd.DataFrame(sdg_mlb,columns=goals)
target_mlb = mlb.fit_transform(targets_main.values)
target_mlb = pd.DataFrame(target_mlb,columns=goals)

sdg_mlb.drop('00',axis=1,inplace=True)
target_mlb.drop('00',axis=1,inplace=True)
goals = np.delete(goals,0)

item_diff = (sdg_mlb-target_mlb)
goal_diff = item_diff.copy().T

def set_diff(A):
    cols = A.columns.get_values()
    s_t = np.array([cols[row==1] for row in A.values])
    t_s = np.array([cols[row==-1] for row in A.values])
    A['sdg-trg cols'] = s_t
    A['sdg-trg count'] = A['sdg-trg cols'].apply(len)
    A['trg-sdg cols'] = t_s
    A['trg-sdg count'] = A['trg-sdg cols'].apply(len)    
    return A
    

item_diff = set_diff(item_diff)
item_diff.index.name = 'item'
display(item_diff)

goal_diff = set_diff(goal_diff)
goal_diff.index.name = 'goals'
display(goal_diff)

,01,02,03,04,05,06,07,08,09,10,...,17,Capacity-Building,Finance,Systemicissues,Technology,Trade,sdg-trg cols,sdg-trg count,trg-sdg cols,trg-sdg count
item,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,"[13, 17]",2,[],0
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,"[03, 07, 13]",3,[],0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,[],0,[],0


,0,1,2,3,4,5,6,7,8,9,...,1374,1375,1376,1377,1378,1379,sdg-trg cols,sdg-trg count,trg-sdg cols,trg-sdg count
goals,,,,,,,,,,,,,,,,,,,,,
01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[19, 45, 50, 68, 79, 102, 105, 110, 111, 116, ...",94,[],0
02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,"[19, 38, 45, 48, 50, 56, 68, 79, 105, 110, 111...",127,[],0
03,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[2, 10, 38, 45, 56, 79, 110, 127, 186, 219, 22...",77,[],0
04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[56, 95, 110, 111, 168, 201, 207, 220, 350, 36...",36,[],0
05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[68, 79, 110, 111, 127, 194, 201, 219, 220, 23...",53,[],0
06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[19, 24, 29, 56, 110, 133, 186, 194, 219, 220,...",60,[],0
07,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[2, 24, 56, 110, 111, 194, 279, 350, 366, 465,...",35,[],0
08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[24, 41, 50, 56, 68, 79, 110, 111, 194, 201, 2...",98,[],0
09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"[24, 56, 110, 111, 194, 201, 254, 279, 286, 29...",51,[],0
